In [1]:
import torch
import monai
from torch.export import export
from executorch.exir import to_edge
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner
from executorch.exir import EdgeCompileConfig

/home/kindersc/Documents/model_conversion/executorch_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
densenet = getattr(monai.networks.nets, 'densenet121')
model = densenet(spatial_dims=2,
                    in_channels=3,
                    out_channels=3,
                    dropout_prob=float(0.2),
                    pretrained=True)

In [3]:
# Load the weights from the file
weights_path = '/home/kindersc/Documents/executorch_example/executorch/notebooks/best_metric_model.pth'
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))

/tmp/ipykernel_168726/579817510.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_location=torch.device('cpu'))


In [4]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [5]:
model.eval()

DenseNet121(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (dropout): Dropout2d(p=0.2, inplace=False)
        )
      )
      (denselayer2): _DenseLayer(
  

In [6]:
import torch
from torch.utils.data import DataLoader, Dataset

# Custom dataset returning an image of all 1s
class OneImageDataset(Dataset):
    def __init__(self):
        self.target = 0  # Dummy target (can be changed if needed)

    def __len__(self):
        return 1000  # Single item in the dataset

    def __getitem__(self, idx):
        image = torch.randn((3, 256, 256))
        return image, self.target

# Define a data loader with batch size 1
data_loader = DataLoader(OneImageDataset(), batch_size=1)

In [7]:
# define calibration function
def calibrate(model_in, data_loader):
    with torch.no_grad():
        for image, target in data_loader:
            model_in(image)

In [8]:
from torch._export import capture_pre_autograd_graph
from torch.ao.quantization.quantize_pt2e import convert_pt2e, prepare_pt2e
from torch.ao.quantization.quantizer.xnnpack_quantizer import (
    get_symmetric_quantization_config,
    XNNPACKQuantizer,
)
from torch.export import export, ExportedProgram

In [9]:
example_args = (torch.ones(1, 3, 256, 256),)
pre_autograd_aten_dialect = capture_pre_autograd_graph(model, example_args)

W0926 19:43:43.120000 168726 torch/_export/__init__.py:64] +============================+
W0926 19:43:43.120000 168726 torch/_export/__init__.py:65] |     !!!   WARNING   !!!    |
W0926 19:43:43.121000 168726 torch/_export/__init__.py:66] +============================+
W0926 19:43:43.121000 168726 torch/_export/__init__.py:67] capture_pre_autograd_graph() is deprecated and doesn't provide any function guarantee moving forward.
W0926 19:43:43.121000 168726 torch/_export/__init__.py:68] Please switch to use torch.export.export_for_training instead.


In [10]:
print("Pre-Autograd ATen Dialect Graph")
print(pre_autograd_aten_dialect)

Pre-Autograd ATen Dialect Graph
GraphModule()



def forward(self, x):
    arg0, = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
    arg0_1 = arg0
    _param_constant0 = self.features_conv0_weight
    conv2d = torch.ops.aten.conv2d.default(arg0_1, _param_constant0, None, [2, 2], [3, 3]);  arg0_1 = _param_constant0 = None
    empty = torch.ops.aten.empty.memory_format([0], dtype = torch.uint8, layout = torch.strided, device = device(type='cpu'));  empty = None
    _param_constant1 = self.features_norm0_weight
    _param_constant2 = self.features_norm0_bias
    _tensor_constant0 = self.features_norm0_running_mean
    _tensor_constant1 = self.features_norm0_running_var
    _native_batch_norm_legit_no_training = torch.ops.aten._native_batch_norm_legit_no_training.default(conv2d, _param_constant1, _param_constant2, _tensor_constant0, _tensor_constant1, 0.1, 1e-05);  conv2d = _param_constant1 = _param_constant2 = _tensor_constant0 = _tensor_constant1 = None
    getitem = _native_batc

In [11]:
quantizer = XNNPACKQuantizer().set_global(get_symmetric_quantization_config())

In [12]:
quantizer

In [13]:
prepared_graph = prepare_pt2e(pre_autograd_aten_dialect, quantizer)

/home/kindersc/Documents/model_conversion/executorch_venv/lib/python3.10/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(_native_batch_norm_legit_no_training) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")
/home/kindersc/Documents/model_conversion/executorch_venv/lib/python3.10/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(_native_batch_norm_legit_no_training_2) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")
/home/kindersc/Documents/model_conversion/executorch_venv/lib/python3.10/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(_native_batch_norm_legit_no_training_4) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")
/home/kindersc/Documents/model_conversion/executorch_venv/lib/python3.10/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(_native_batch_norm_legit_no_training_6) on an already era

In [14]:
prepared_graph

GraphModule(
  (activation_post_process_0): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_1): MinMaxObserver(min_val=inf, max_val=-inf)
  (activation_post_process_2): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_3): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_4): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_5): MinMaxObserver(min_val=inf, max_val=-inf)
  (activation_post_process_6): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_7): MinMaxObserver(min_val=inf, max_val=-inf)
  (activation_post_process_8): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_9): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_10): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_11): MinMaxObserver(min_val=inf, max_val=-inf)
  (activation_post_process_12): HistogramObserver(min_val=inf, max_val=-inf)
  (activ

In [15]:
calibrate(prepared_graph, data_loader)

In [16]:
# calibrate with a sample dataset
converted_graph = convert_pt2e(prepared_graph)

In [17]:
print("Quantized Graph")
print(converted_graph)

Quantized Graph
GraphModule()



def forward(self, x):
    arg0, = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
    arg0_1 = arg0
    quantize_per_tensor_default = torch.ops.quantized_decomposed.quantize_per_tensor.default(arg0_1, 0.0309614110738039, 0, -128, 127, torch.int8);  arg0_1 = None
    dequantize_per_tensor_default = torch.ops.quantized_decomposed.dequantize_per_tensor.default(quantize_per_tensor_default, 0.0309614110738039, 0, -128, 127, torch.int8);  quantize_per_tensor_default = None
    _frozen_param0 = self._frozen_param0
    dequantize_per_tensor_default_1 = torch.ops.quantized_decomposed.dequantize_per_tensor.default(_frozen_param0, 0.01635424979031086, 0, -127, 127, torch.int8);  _frozen_param0 = None
    features_conv0_weight_bias = self.features_conv0_weight_bias
    conv2d = torch.ops.aten.conv2d.default(dequantize_per_tensor_default, dequantize_per_tensor_default_1, features_conv0_weight_bias, [2, 2], [3, 3]);  dequantize_per_tensor_default = dequantize_p

In [18]:
aten_dialect: ExportedProgram = export(converted_graph, example_args)

In [19]:
print("ATen Dialect Graph")
print(aten_dialect)

ATen Dialect Graph
ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_features_conv0_weight_bias: "f32[64]", p_features_denseblock1_denselayer1_layers_norm1_weight: "f32[64]", p_features_denseblock1_denselayer1_layers_norm1_bias: "f32[64]", p_features_denseblock1_denselayer1_layers_conv1_weight_bias: "f32[128]", p_features_denseblock1_denselayer2_layers_norm1_weight: "f32[96]", p_features_denseblock1_denselayer2_layers_norm1_bias: "f32[96]", p_features_denseblock1_denselayer2_layers_conv1_weight_bias: "f32[128]", p_features_denseblock1_denselayer3_layers_norm1_weight: "f32[128]", p_features_denseblock1_denselayer3_layers_norm1_bias: "f32[128]", p_features_denseblock1_denselayer3_layers_conv1_weight_bias: "f32[128]", p_features_denseblock1_denselayer4_layers_norm1_weight: "f32[160]", p_features_denseblock1_denselayer4_layers_norm1_bias: "f32[160]", p_features_denseblock1_denselayer4_layers_conv1_weight_bias: "f32[128]", p_features_denseblock1_denselayer

In [20]:
# # 1. torch.export: Defines the program with the ATen operator set.
# aten_dialect = export(model, (torch.ones(1, 3, 256, 256),))

In [21]:
# 2. to_edge: Make optimizations for Edge devices
edge_program = to_edge(aten_dialect, compile_config=EdgeCompileConfig(_check_ir_validity=False))

In [22]:
edge_program = edge_program.to_backend(XnnpackPartitioner())

In [23]:
# 3. to_executorch: Convert the graph to an ExecuTorch program
executorch_program = edge_program.to_executorch()

In [24]:
# 4. Save the compiled .pte program
with open("model_threesix_quantized.pte", "wb") as file:
    file.write(executorch_program.buffer)